<a href="https://colab.research.google.com/github/adalves-ufabc/2022.Q2-PLN/blob/main/2022_Q2_PLN_Notebook_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2022.Q2]**

Prof. Alexandre Donizeti Alves

##**Modelo de Linguagem com N-gramas**


### **Bibliotecas**


In [ ]:
# expressoes regulares
import re

# cria um dicinario com a frequencia dos termos em um iterable
from collections import Counter

## subsequencias de um iterable
from itertools import islice

# numeros e sequencias aleatoreas
import random

### **Funções principais**

In [ ]:
regex = r"[-'a-zA-ZÀ-ÖØ-öø-ÿ]+"       # raw string
#regex = r"[-'a-zA-ZÀ-ÖØ-öø-ÿ0-9]+"   # raw string

# usa expressoes regulares para quebrar um texto em tokens
def get_tokens(fileName):
  # leitura do documento
  with open(fileName,'r') as document:
     content  = document.read()  # devolve um vetor contendo as linhas do arquivo
     content  = content.lower()

  Words    = re.findall(regex, content)

  return(Words)

# similar ao get_tokens, mas removindo elementos da lista de stopwords
def get_tokens_without_stopwords(fileName,stopwordsName="/content/stopwords.txt"):
   # leitura do documento
   with open(fileName,'r') as document:
      content  = document.read()  # devolve um vetor contendo as linhas do arquivo
      content  = content.lower()

   # leitura das stopwords
   with open(stopwordsName,'r') as stopwordsfile:
      stopwords = set([])
      for s in stopwordsfile.readlines():                                                                                                                                                     
        stopwords.add(s.strip().lower())

   # remove as stopwords
   Words    = [w for w in re.findall(regex, content) if w not in stopwords]

   return(Words)

# retirado de um exemplo na internet
def window(seq, n=2):
    "Retorna uma janela deslizante (de tamanho n) sobre a sequencia seq"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result    
    for elem in it:
        result = result[1:] + (elem,)
        yield result

# isa o Counter para contar a frequencia de unigramas e bigramas
def ngrams(Words):
    "Retarna a contagem de unigramas e bigramas a partir da lista de palavras"
  
    # Conta Unigramas (utilizando o counter de collections)
    Unigrams = Counter(Words)

    # windows retora uma janela delizante de tamanho 2
    Bigrams  = Counter(window(Words,2))

    return(Unigrams,Bigrams)

# funcao auxiliar para calcular as probabilidades
BigramProbabilities = lambda w1, w2 : bigrams [ (w1,w2) ] / unigrams[ w1 ]

# aplica o score em uma lista de sentencas
def score(phrases):

    # loop sobre todas as sentencas de teste
    for phrase in phrases:
        Words = re.findall(regex, phrase)
        P = float(1.0)
        for w_0, w_1 in window(Words,2):
            P = P * BigramProbabilities(w_0, w_1)

        print ( "{1:.20f} : Sentença: {0}".format( phrase, P ) ) 

### **Testando o Modelo**

Cria uma sequência de sentenças e um modelo de bigramas baseado no livro de **Machado de Assis**, e aplica a função score na lista de sentenças

In [ ]:
sentencas = ["ele é",
             "ele é uma", 
             "ele é uma pessoa", 
             "ele é uma pessoa de", 
             "ele é uma pessoa de verdade"]

words = get_tokens("/content/A-Semana-Machado-de-Assis.txt")

unigrams, bigrams = ngrams(words)
score(sentencas)

0.02647657841140529586 : Sentença: ele é
0.00069189664838167149 : Sentença: ele é uma
0.00000842133213707000 : Sentença: ele é uma pessoa
0.00000050654629395910 : Sentença: ele é uma pessoa de
0.00000000051863038186 : Sentença: ele é uma pessoa de verdade


Repetimos o processo, agora considerando todas as obras de Machado de Assis

In [ ]:
sentencas = ["ele é",
             "ele é uma", 
             "ele é uma pessoa", 
             "ele é uma pessoa de", 
             "ele é uma pessoa de verdade"]

words = get_tokens("/content/Todas-as-obras-Machado-de-Assis.txt")

unigrams, bigrams = ngrams(words)
score(sentencas)

0.02488038277511961729 : Sentença: ele é
0.00093406050182903407 : Sentença: ele é uma
0.00000825483405278844 : Sentença: ele é uma pessoa
0.00000046590235609081 : Sentença: ele é uma pessoa de
0.00000000050723310908 : Sentença: ele é uma pessoa de verdade


In [ ]:
print(len(words))

1535663


In [ ]:
print(len(unigrams))

63328


In [ ]:
print(len(bigrams))

618262


### **Probabilidade da próxima palavra**

Usamos o modelo de bigrama para calcular quais as palavras mais prováveis.

In [ ]:
def next_prob(phrase,n=5):
    # quebre a sentenca em palavras
    Words = re.findall(regex, phrase)

    # calcula as probabilidades de todas as palavras em que o bigrama eh w1 e armazena em prob
    probs = {w2 : BigramProbabilities(w1,w2) for (w1,w2) in bigrams.keys() if w1 == Words[-1] }

    # ordena e imprime as n mais relevantes
    for w, p in islice(sorted(probs.items(), key = lambda item: item[1], reverse=True),n):
        print ( "{0} -> {1} ({2:.2f}%)".format( phrase, w.upper(), p*100 ) )   

**Teste da função `next_prob`**

O usuário digita frases e são listadas as 5 palavras mais prováveis de acordo com o modelo.

In [ ]:
words = get_tokens("/content/A-Semana-Machado-de-Assis.txt")
#words = get_tokens("/content/Todas-as-obras-Machado-de-Assis.txt")

unigrams, bigrams = ngrams(words)

phrase = input("\nDigite uma frase: ")

while (phrase != ""):
    next_prob(phrase)
    phrase = input("\nDigite uma frase:")

# frase: ele é uma pessoa
# frase: estudar
# frase: a semana que


Digite uma frase: ele é uma pessoa
ele é uma pessoa -> QUE (28.57%)
ele é uma pessoa -> DE (6.02%)
ele é uma pessoa -> A (4.51%)
ele é uma pessoa -> É (2.26%)
ele é uma pessoa -> NO (2.26%)

Digite uma frase:estudar
estudar -> A (30.00%)
estudar -> O (30.00%)
estudar -> E (10.00%)
estudar -> ESSA (10.00%)
estudar -> AS (10.00%)

Digite uma frase:a semana que
a semana que -> O (7.33%)
a semana que -> A (6.90%)
a semana que -> NÃO (5.89%)
a semana que -> SE (4.82%)
a semana que -> É (3.59%)

Digite uma frase:


**Teste da função `next_prob`**

O usuário digita frases e são listadas as 5 palavras mais prováveis de acordo com o modelo, **sem considerar as *stopwords***

In [ ]:
words = get_tokens_without_stopwords("/content/A-Semana-Machado-de-Assis.txt")
#words = get_tokens_without_stopwords("/content/Todas-as-obras-Machado-de-Assis.txt")

unigrams, bigrams = ngrams(words)

phrase = input("\nDigite uma frase: ")

while (phrase != ""):
    next_prob(phrase)
    phrase = input("\nDigite uma frase:")

# frase: que
# frase: rio


Digite uma frase: que

Digite uma frase:rio
rio -> JANEIRO (54.70%)
rio -> GRANDE (15.38%)
rio -> BRANCO (5.98%)
rio -> CLARO (2.56%)
rio -> NEWS (1.71%)

Digite uma frase:
